import section

In [ ]:
import pandas as pd
from pandas import DataFrame
import ast
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
# Load the sales data
sales_df = pd.read_csv('data/sale.csv', sep=',')

# Load the menu data
menu_df = pd.read_csv('data/menus.csv', sep=',')

# Load the add-on data
add_on_df = pd.read_csv('data/add-on.csv', sep=',')

# Check basic info of each dataframe
menu_df.info(), sales_df.info(), add_on_df.info()

Additional function

In [ ]:
# Function to calculate the total price of add-ons
def calculate_addon_price(addons, addons_df: DataFrame) -> float:
    addons_list = ast.literal_eval(addons)  # Convert the string representation of list to actual list
    total_price = 0
    for addon in addons_list:
        price = addons_df.loc[addons_df['name'] == addon, 'price']
        if not price.empty:
            total_price += price.values[0]
    return total_price

Preparation section

In [ ]:
# Preprocess menu_df
menu_df['price'] = menu_df['price'].astype(float)  # Ensure price is of type float
menu_df = menu_df.drop(columns=['uuid', 'description'])  # Drop unnecessary columns
menu_df = menu_df.rename(columns={'name': 'menu_name'})  # Rename column for consistency
menu_df = menu_df.dropna(axis=0, how='any')  # Drop rows with any NaN values
menu_df.info()

In [ ]:
# Preprocess sales_df
sales_df = sales_df.drop(columns=['uuid'])  # Drop unnecessary columns
sales_df = sales_df.rename(columns={'menu_item': 'menu_name'})  # Rename column for consistency
sales_df = sales_df.sort_values('date_time', ascending=True).reset_index(drop=True)  # Sort by date_time and reset index
sales_df = sales_df.dropna(axis=0, how='any')  # Drop rows with any NaN values
sales_df.info()

In [ ]:
# Preprocess add_on_df
add_on_df = add_on_df.drop(columns=['uuid'])  # Drop unnecessary columns
add_on_df = add_on_df.dropna(axis=0, how='any')  # Drop rows with any NaN values
add_on_df.info()

In [ ]:
# Merge dataframes
merged_df = pd.merge(menu_df, sales_df, on='menu_name', how='inner')  # Merge menu and sales data on menu_name
merged_df = merged_df.dropna(axis=0, how='any')  # Drop rows with any NaN values
merged_df["date_time"] = pd.to_datetime(merged_df["date_time"])  # Convert date_time to datetime format
merged_df = merged_df.sort_values(by='date_time', ascending=True).reset_index(
    drop=True)  # Sort by date_time and reset index
merged_df["date"] = merged_df["date_time"].dt.strftime("%Y-%m-%d")  # Extract date
merged_df["time"] = merged_df["date_time"].dt.strftime("%H:%M")  # Extract time
# merged_df = merged_df.drop(columns=["date_time"])  # Drop the original date_time column
merged_df["add_ons_price"] = merged_df["add_ons"].apply(
    lambda x: calculate_addon_price(x, add_on_df))  # Calculate add_ons_price
merged_df["total_price"] = merged_df["add_ons_price"] + merged_df["price"]  # Calculate total price
merged_df.info()

In [ ]:
# Check for missing values
missing_values = merged_df.isnull().sum()
print(missing_values)

In [ ]:
# Display the first few rows of the final merged dataframe
merged_df.head(20)

**Analysis section**

---

In [ ]:
# Analyze merged_df for negative prices
negative_price = merged_df[merged_df['price'] < 0]
negative_add_ons_price = merged_df[merged_df['add_ons_price'] < 0]
negative_total_price = merged_df[merged_df['total_price'] < 0]

# Display dataframes with negative values
print(negative_price)
print(negative_add_ons_price)
print(negative_total_price)

In [ ]:
# Display first few rows and summary statistics of add_on_df
add_on_df.head()

In [ ]:
add_on_df.describe()

In [ ]:
# Count occurrences of each menu in add_on_df
menu_counts = add_on_df['menu']
menu_counts.value_counts()

In [ ]:
# Plot the distribution of add-on prices
plt.figure(figsize=(10, 6))
plt.hist(add_on_df['price'], bins=10, edgecolor='black')
plt.title('Price Distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Calculate and plot average price per menu item in add_on_df
average_price_per_menu = add_on_df.groupby('menu')['price'].mean()
plt.figure(figsize=(10, 6))
average_price_per_menu.plot(kind='bar')
plt.title('Average Price per Menu Item')
plt.xlabel('Menu')
plt.ylabel('Average Price')
plt.show()

In [ ]:
# Add day of the week to merged_df
merged_df['date'] = pd.to_datetime(merged_df['date'])  # Ensure 'date' is in datetime format
merged_df['day_of_week'] = merged_df['date'].dt.day_name()  # Extract day of the week

In [ ]:
merged_df.head()

In [ ]:
merged_df.describe()

In [ ]:
# Plot the distribution of total prices in merged_df
plt.figure(figsize=(10, 6))
plt.hist(merged_df['total_price'], bins=50, edgecolor='black')
plt.xlabel('Total Price')
plt.ylabel('Frequency')
plt.title('Distribution of Total Price')
plt.show()

In [ ]:
# Perform ANOVA to test for significant differences in total_price between categories
categories = merged_df['category'].unique()
data = [merged_df[merged_df['category'] == category]['total_price'] for category in categories]
f_val, p_val = stats.f_oneway(*data)
print(f'ANOVA results: F-value = {f_val}, P-value = {p_val}')

In [ ]:
 # Calculate and plot average total price by category
avg_price_by_category = merged_df.groupby('category')['total_price'].mean()
plt.figure(figsize=(10, 6))
plt.bar(avg_price_by_category.index, avg_price_by_category.values)
plt.xlabel('Category')
plt.ylabel('Average Total Price')
plt.title('Average Total Price by Category')
plt.xticks(rotation=90)
plt.show()

# analyze sale

In [ ]:
sale_analyze_df = merged_df

In [ ]:
sale_analyze_df.head()

In [ ]:
# Calculate sales for each item
sale_analyze_df['sales'] = sale_analyze_df['price'] * sale_analyze_df.groupby('menu_name')['menu_name'].transform(
    'count')

# Total sales
total_sales = sale_analyze_df['sales'].sum()
print(f'Total estimated sales: ${total_sales:.2f}')

In [ ]:
# Define time categories
def categorize_time(hour):
    if 6 <= hour < 11:
        return 'Breakfast'
    elif 11 <= hour < 14:
        return 'Lunch'
    elif 14 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Dinner'
    else:
        return 'Other'

In [ ]:
# Function to plot bars for aggregated data
def plot_bars(agg_data, title, x_label, y_label):
    agg_data.plot(kind='bar', figsize=(12, 6))
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Extract year, month, and day from date
sale_analyze_df['year'] = sale_analyze_df['date_time'].dt.year
sale_analyze_df['month'] = sale_analyze_df['date_time'].dt.month
sale_analyze_df['day'] = sale_analyze_df['date_time'].dt.day
sale_analyze_df['hour'] = sale_analyze_df['date_time'].dt.hour

# Calculate total sales, total cost, and total sale difference (sales - cost) for all data
total_sales_all = sale_analyze_df['price'].sum()
total_cost_all = sale_analyze_df['cost'].sum()
total_sale_difference_all = total_sales_all - total_cost_all

# Calculate total sales, total cost, and sale difference (sales - cost)
sale_analyze_df['total_sales'] = sale_analyze_df.groupby(['year', 'month', 'day'])['price'].transform('sum')
sale_analyze_df['total_cost'] = sale_analyze_df.groupby(['year', 'month', 'day'])['cost'].transform('sum')
sale_analyze_df['total_sale_difference'] = sale_analyze_df['total_sales'] - sale_analyze_df['total_cost']
sale_analyze_df['sale_difference'] = sale_analyze_df['price'] - sale_analyze_df['cost']

sale_analyze_df['time_category'] = sale_analyze_df['hour'].apply(categorize_time)

# Calculate sales and sale difference
sale_analyze_df['sales'] = sale_analyze_df['price']
sale_analyze_df['sale_difference'] = sale_analyze_df['sales'] - sale_analyze_df['cost']

In [ ]:
# Aggregate by day of week
agg_day_of_week = sale_analyze_df.groupby('day_of_week').agg({
    'sales': 'sum',
    'cost': 'sum',
    'sale_difference': 'sum'
}).reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

# Aggregate by time category
agg_time_category = sale_analyze_df.groupby('time_category').agg({
    'sales': 'sum',
    'cost': 'sum',
    'sale_difference': 'sum'
})

# Aggregate by year, month, day (as previously described)
agg_year = sale_analyze_df.groupby('year').agg({
    'sales': 'sum',
    'cost': 'sum',
    'sale_difference': 'sum'
}).reset_index()

agg_month = sale_analyze_df.groupby(['year', 'month']).agg({
    'sales': 'sum',
    'cost': 'sum',
    'sale_difference': 'sum'
}).reset_index()

agg_month_avg = sale_analyze_df.groupby('month').agg({
    'sales': 'mean',
    'cost': 'mean',
    'sale_difference': 'mean'
}).reset_index()

agg_day = sale_analyze_df.groupby(['year', 'month', 'day']).agg({
    'sales': 'sum',
    'cost': 'sum',
    'sale_difference': 'sum'
}).reset_index()

agg_day_avg = sale_analyze_df.groupby(['month', 'day']).agg({
    'sales': 'mean',
    'cost': 'mean',
    'sale_difference': 'mean'
}).reset_index()

# Plot total sales, cost, and sale difference for all data
total_sales_all = sale_analyze_df['sales'].sum()
total_cost_all = sale_analyze_df['cost'].sum()
total_sale_difference_all = total_sales_all - total_cost_all


In [ ]:
plt.figure(figsize=(10, 6))
labels = ['Total Sales', 'Total Cost', 'Total Sale Difference']
values = [total_sales_all, total_cost_all, total_sale_difference_all]
plt.bar(labels, values, color=['blue', 'red', 'green'])
plt.title('Total Sales, Cost, and Sale Difference for All Data')
plt.ylabel('Amount')
plt.tight_layout()
plt.show()

In [ ]:
# Plot sales, cost, and sale difference by year
plot_bars(agg_year.set_index('year'), 'Sales, Cost, and Sale Difference by Year', 'Year', 'Amount')

In [ ]:
# Plot sales, cost, and sale difference by month (total in each month)
plot_bars(agg_month.pivot(index='month', columns='year', values='sales'), 'Sales by Month', 'Month', 'Total Sales')

In [ ]:
plot_bars(agg_month.pivot(index='month', columns='year', values='cost'), 'Cost by Month', 'Month', 'Total Cost')

In [ ]:
plot_bars(agg_month.pivot(index='month', columns='year', values='sale_difference'), 'Sale Difference by Month', 'Month', 'Total Sale Difference')

In [ ]:
# Plot average sales, cost, and sale difference by month
plot_bars(agg_month_avg.set_index('month'), 'Average Sales, Cost, and Sale Difference by Month', 'Month', 'Average Amount')

In [ ]:

# Plot sales, cost, and sale difference by day (total in each day)
plot_bars(agg_day.set_index(['year', 'month', 'day']), 'Sales, Cost, and Sale Difference by Day', 'Day', 'Amount')

In [ ]:
# Plot average sales, cost, and sale difference by day
plot_bars(agg_day_avg.set_index(['month', 'day']), 'Average Sales, Cost, and Sale Difference by Day', 'Day', 'Average Amount')

In [ ]:
# Plot sales, cost, and sale difference by day of week
plot_bars(agg_day_of_week, 'Sales, Cost, and Sale Difference by Day of Week', 'Day of Week', 'Amount')

In [ ]:
# Plot sales, cost, and sale difference by time category
plot_bars(agg_time_category, 'Sales, Cost, and Sale Difference by Time Category', 'Time Category', 'Amount')

In [ ]:
# Count number of customers per day of the week
daily_customers = sale_analyze_df.groupby('day_of_week').size()

plt.figure(figsize=(10, 6))
daily_customers.plot(kind='bar', color='skyblue')
plt.title('Number of Customers by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Optional: Create time categories (e.g., breakfast, lunch, dinner)
# Define time intervals for categorization
bins = [0, 6, 12, 18, 24]
labels = ['Night', 'Morning', 'Afternoon', 'Evening']
sale_analyze_df['time_category'] = pd.cut(sale_analyze_df['date_time'].dt.hour, bins=bins, labels=labels, right=False)

# Count number of customers by time category
time_customers = sale_analyze_df.groupby('time_category').size()

# Plotting number of customers by time category
plt.figure(figsize=(8, 5))
time_customers.plot(kind='bar', color='lightgreen')
plt.title('Number of Customers by Time of Day')
plt.xlabel('Time of Day')
plt.ylabel('Number of Customers')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()